In [8]:
import torch
from pathlib import Path

from models.lit_lraspp_mobilenet_v3_large import LRASPPMobileNetV3Large
def train():
    pass

DEVICE = torch.device('cpu')


In [2]:
weights_path = "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa/LRASPP/best-val_miou=0.8023-epoch=18-step=17593.ckpt"
torchscript_path = "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa/LRASPP/LRASPP_MobileNetV3_kelp_presence_trt_jit.pt"
model = LRASPPMobileNetV3Large.load_from_checkpoint(weights_path, train=False, map_location=DEVICE, strict=False, num_classes=2)
# model.to_torchscript(file_path=torchscript_path, method='trace', example_inputs=x)

In [3]:
# Save state_dict only
torch.save(model.state_dict(), Path(weights_path).with_suffix(".pt"))

In [8]:

x = torch.rand(1, 3, 8, 8, device=DEVICE, requires_grad=False)

model.eval()
model.requires_grad_(False)
with torch.no_grad():
    traced_model = torch.jit.trace(model, example_inputs=x)
    traced_model.save(torchscript_path)
# torch.onnx.export(model, x, "torchscript_files/LRASPP_MobileNetV3_kelp_species.onnx",
#                   opset_version=11, verbose=True)

/home/taylor/anaconda3/envs/hakai-ml-train/envs/hakai-ml-train/lib/python3.9/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: The `LightningModule.model_size` property was deprecated in v1.5 and will be removed in v1.7. Please use the `pytorch_lightning.utilities.memory.get_model_size_mb`.
  if hasattr(mod, name):
/home/taylor/anaconda3/envs/hakai-ml-train/envs/hakai-ml-train/lib/python3.9/site-packages/torch/_jit_internal.py:669: LightningDeprecationWarning: The `LightningModule.model_size` property was deprecated in v1.5 and will be removed in v1.7. Please use the `pytorch_lightning.utilities.memory.get_model_size_mb`.
  item = getattr(mod, name)
/home/taylor/anaconda3/envs/hakai-ml-train/envs/hakai-ml-train/lib/python3.9/site-packages/torch/_jit_internal.py:668: LightningDeprecationWarning: `LightningModule.use_amp` was deprecated in v1.6 and will be removed in v1.8. Please use `Trainer.amp_backend`.
  if hasattr(mod, name):
/home/taylor/anaconda3/envs/hakai-ml

In [65]:
weights_paths = [
    "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa_aco/lr_aspp_kelp_pa_aco/checkpoints/val_miou=06707.ckpt",
    "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa_aco/lr_aspp_kelp_pa_aco/checkpoints/val_miou=06705.ckpt",
    "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa_aco/lr_aspp_kelp_pa_aco/checkpoints/val_miou=06703.ckpt",
]
torchscript_path = "/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/kelp_pa_aco/lr_aspp_kelp_pa_aco/checkpoints/LRASPP_MobileNetV3_kelp_presence_aco_jit.pt"
models = [
    LRASPPMobileNetV3Large.load_from_checkpoint(weights_path, train=False, map_location=DEVICE, strict=False, num_classes=2)
    for weights_path in weights_paths
]

class Ensemble(torch.nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = models
        for m in self.models:
            m.requires_grad_(False)
            m.eval()

    def forward(self, x):
        probs = torch.sum(torch.stack([torch.softmax(m(x), dim=1) for m in self.models]), dim=1)
        return probs.detach().cpu()

model = Ensemble(models)

In [66]:
x = torch.rand(1, 3, 8, 8, device=DEVICE, requires_grad=False)

model.eval()
model.requires_grad_(False)
with torch.no_grad():
    traced_model = torch.jit.trace(model, example_inputs=x)
    traced_model.save(torchscript_path)
# torch.onnx.export(model, x, "torchscript_files/LRASPP_MobileNetV3_kelp_species.onnx",
#                   opset_version=11, verbose=True)

In [33]:
# torch.onnx.export(model, x, onnx_path,
#                   input_names=['input'],
#                   output_names=['output'],
#                   dynamic_axes={
#                       'input': {
#                           0: 'batch_size',
#                           2: 'width',
#                           3: 'length'
#                       },
#                       'output': {
#                           0: 'batch_size',
#                           2: 'width',
#                           3: 'length'
#                       }
#                   })

NameError: name 'onnx_path' is not defined

In [9]:
# % pip install onnx onnxruntime-gpu

Note: you may need to restart the kernel to use updated packages.


In [10]:
# import onnx
#
# onnx_model = onnx.load(onnx_path)
# onnx.checker.check_model(onnx_model)

In [11]:
# import onnxruntime as ort
# import numpy as np
#
# z = np.ones((1, 3, 128, 128)).astype(np.float32)
# print(z.dtype)
# ort_sess = ort.InferenceSession(onnx_path,
#                                 providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider',
#                                            'CPUExecutionProvider'])
# outputs = ort_sess.run(None, {'input': z})

float32


In [50]:
print(np.argmax(np.asarray(outputs[0]), axis=1))

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [51]:
z - np.expand_dims(np.array([0.1, 0.2, 0.3]), axis=[1, 2])

array([[[[0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         ...,
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9],
         [0.9, 0.9, 0.9, ..., 0.9, 0.9, 0.9]],

        [[0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         ...,
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8],
         [0.8, 0.8, 0.8, ..., 0.8, 0.8, 0.8]],

        [[0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         ...,
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7],
         [0.7, 0.7, 0.7, ..., 0.7, 0.7, 0.7]]]])

In [61]:
from torch import package

path = '/home/taylor/PycharmProjects/hakai-ml-train/checkpoints/mussels/lraspp_mussels.pt'
package_name = 'lraspp_mussels'
resource_name = "model.pkl"

with package.PackageExporter(path) as exp:
    exp.intern(["lit_lraspp_mobilenet_v3_large", "utils.**", "kelp_data_module",
                "geotiff_crop_dataset.**"])
    exp.extern(
        ["pytorch_lightning.**", "torchmetrics.**", "torchvision.**", "numpy", "PIL.**", "tqdm.**",
         "rasterio"])
    exp.save_pickle(package_name, resource_name, model)